## Imports

In [1]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
import tensorflow_datasets as tfds

from keras.models import Sequential

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


## Load and prepare the dataset

The [CIFAR 10](https://www.tensorflow.org/datasets/catalog/cifar10) dataset already has train and test splits and you can use those in this exercise. Here are the general steps:

* Load the train/test split from TFDS. Set `as_supervised` to `True` so it will be convenient to use the preprocessing function we provided.
* Normalize the pixel values to the range [0,1], then return `image, image` pairs for training instead of `image, label`. This is because you will check if the output image is successfully regenerated after going through your autoencoder.
* Shuffle and batch the train set. Batch the test set (no need to shuffle).


In [2]:
# preprocessing function
def map_image(image, label):
  image = tf.cast(image, dtype=tf.float32)
  image = image / 255.0

  return image, image # dataset label is not used. replaced with the same image input.

# parameters
BATCH_SIZE = 128
SHUFFLE_BUFFER_SIZE = 1024


### START CODE HERE (Replace instances of `None` with your code) ###

# use tfds.load() to fetch the 'train' split of CIFAR-10
train_dataset = tfds.load('cifar10', as_supervised = True, split = 'train')

# preprocess the dataset with the `map_image()` function above
train_dataset = train_dataset.map(map_image)

# shuffle and batch the dataset
train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE).repeat()


# use tfds.load() to fetch the 'test' split of CIFAR-10
test_dataset = tfds.load('cifar10', as_supervised = True, split = 'test')

# preprocess the dataset with the `map_image()` function above
test_dataset = test_dataset.map(map_image)

# batch the dataset
test_dataset = test_dataset.batch(BATCH_SIZE).repeat()

### END CODE HERE ###

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cifar10/3.0.2.incompleteP5503Z/cifar10-train.tfrecord*...:   0%|          …

Generating test examples...:   0%|          | 0/10000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cifar10/3.0.2.incompleteP5503Z/cifar10-test.tfrecord*...:   0%|          |…

Dataset cifar10 downloaded and prepared to /root/tensorflow_datasets/cifar10/3.0.2. Subsequent calls will reuse this data.


In [ ]:
# suggested layers to use. feel free to add or remove as you see fit.
# use the Sequential API (you can remove if you want to use the Functional API)
# use `model.add()` to add layers (if using the Sequential API)

### START CODE HERE ###
inputs = tf.keras.layers.Input(shape = (32, 32, 3))
x = tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu', padding = 'same')(inputs)
x = tf.keras.layers.MaxPooling2D(pool_size = (2, 2))(x)

x = tf.keras.layers.Conv2D(64, (3, 3), activation = 'relu', padding = 'same')(x)
x = tf.keras.layers.MaxPooling2D(pool_size = (2, 2))(x)

x = tf.keras.layers.Conv2D(128, (3, 3), activation = 'relu', padding = 'same')(x)
x = tf.keras.layers.MaxPooling2D(pool_size = (2, 2))(x)

bottle_neck = tf.keras.layers.Conv2D(256, (3, 3), activation = 'relu', padding = 'same')(x)
encoder_vis = tf.keras.layers.Conv2D(3, (3, 3), activation = 'relu', padding = 'same')(bottle_neck)

x = tf.keras.layers.Conv2D(128, (3, 3), activation = 'relu', padding = 'same')(bottle_neck)
x = tf.keras.layers.UpSampling2D(size = (2, 2))(x)

x = tf.keras.layers.Conv2D(64, (3, 3), activation = 'relu', padding = 'same')(x)
x = tf.keras.layers.UpSampling2D(size = (2, 2))(x)

x = tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu', padding = 'same')(x)
x = tf.keras.layers.UpSampling2D(size = (2, 2))(x)

outputs = tf.keras.layers.Conv2D(3, (3, 3), activation = 'sigmoid', padding = 'same')(x)

model = tf.keras.Model(inputs, outputs)
### END CODE HERE ###

model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 8, 8, 64)          0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 8, 8, 128)         73856     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 4, 4, 128)         0   

In [ ]:
# Please do not change the model.compile() parameters
model.compile(optimizer='adam', metrics=['accuracy'], loss='mean_squared_error')

In [ ]:
# parameters (feel free to change this)
train_steps = 50000 // BATCH_SIZE
val_steps = 10000 // BATCH_SIZE

### START CODE HERE ###
model.fit(train_dataset, steps_per_epoch = train_steps, validation_data = test_dataset, validation_steps = val_steps, epochs = 50)
### END CODE HERE ###

Epoch 1/50
390/390 [==============================] - 45s 30ms/step - loss: 0.0281 - accuracy: 0.5492 - val_loss: 0.0106 - val_accuracy: 0.7206
Epoch 2/50
390/390 [==============================] - 8s 20ms/step - loss: 0.0101 - accuracy: 0.7281 - val_loss: 0.0084 - val_accuracy: 0.7630
Epoch 3/50
390/390 [==============================] - 8s 20ms/step - loss: 0.0083 - accuracy: 0.7545 - val_loss: 0.0072 - val_accuracy: 0.7670
Epoch 4/50
390/390 [==============================] - 7s 19ms/step - loss: 0.0072 - accuracy: 0.7622 - val_loss: 0.0065 - val_accuracy: 0.7827
Epoch 5/50
390/390 [==============================] - 8s 20ms/step - loss: 0.0065 - accuracy: 0.7679 - val_loss: 0.0059 - val_accuracy: 0.7829
Epoch 6/50
390/390 [==============================] - 8s 20ms/step - loss: 0.0059 - accuracy: 0.7727 - val_loss: 0.0056 - val_accuracy: 0.7574
Epoch 7/50
390/390 [==============================] - 7s 19ms/step - loss: 0.0056 - accuracy: 0.7745 - val_loss: 0.0053 - val_accuracy: 0.785

In [ ]:
result = model.evaluate(test_dataset, steps=10)

10/10 [==============================] - 0s 8ms/step - loss: 0.0027 - accuracy: 0.8179


In [ ]:
def display_one_row(disp_images, offset, shape=(28, 28)):
  '''Display sample outputs in one row.'''
  for idx, noisy_image in enumerate(disp_images):
    plt.subplot(3, 10, offset + idx + 1)
    plt.xticks([])
    plt.yticks([])
    noisy_image = np.reshape(noisy_image, shape)
    plt.imshow(noisy_image, cmap='gray')


def display_results(disp_input_images, disp_encoded, disp_predicted, enc_shape=(8,4)):
  '''Displays the input, encoded, and decoded output values.'''
  plt.figure(figsize=(15, 5))
  display_one_row(disp_input_images, 0, shape=(28,28,))
  display_one_row(disp_encoded, 10, shape=enc_shape)
  display_one_row(disp_predicted, 20, shape=(28,28,))

In [ ]:
# take 1 batch of the dataset
test_dataset = test_dataset.take(1)

# take the input images and put them in a list
output_samples = []
for input_image, image in tfds.as_numpy(test_dataset):
      output_samples = input_image

# pick 10 indices
idxs = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

# prepare test samples as a batch of 10 images
conv_output_samples = np.array(output_samples[idxs])
conv_output_samples = np.reshape(conv_output_samples, (10, 28, 28, 1))

# get the encoder ouput
encoded = convolutional_encoder_model.predict(conv_output_samples)

# get a prediction for some values in the dataset
predicted = convolutional_model.predict(conv_output_samples)

# display the samples, encodings and decoded values!
display_results(conv_output_samples, encoded, predicted, enc_shape=(7,7))

<img src="https://drive.google.com/uc?export=view&id=12Fy-guiP-3tTPfc9IV2nHhqLvs7LwRo6" width="75%" height="75%"/>

In [ ]:
model.save('mymodel.h5')